# Import module 

In [1]:
from minio import Minio
from minio.error import S3Error
import os

Create Minio object

In [2]:
client = Minio(
        endpoint="minio:9000",
        access_key="phonghuynh",
        secret_key="phonghuynh",
        secure=False
    )

# Check buckets / objects

### Check exist bucket function

In [54]:
#Check bucket, if not exist -> create
def check_bucket_exist(client, bucket: str) -> None:
    if client.bucket_exists(bucket):
        print(f"bucket {bucket} exits")
    else:
        print(f"bucket {bucket} does not exist")
        print(f"Create bucket {bucket}")
        client.make_bucket(bucket)


Check bucket name "warehouse-manual"

In [4]:
bucket = "warehouse-manual"
check_bucket_exist(client, bucket)

bucket warehouse-manual exits


list all bucket

In [5]:
buckets = client.list_buckets()
print(buckets)

[Bucket('warehouse-manual'), Bucket('warehouse-script')]


## Check object in bucket

Check object in bucket `warehouse-manual`

In [6]:
objects_manual = client.list_objects("warehouse-manual")
print([obj.object_name for obj in objects_manual])

['4.PNG', 'Largest-Bank.csv', 'bank_market_cap.json', 'download.jpeg']


Check object in bucket `warehouse-script`

In [7]:
objects_script = client.list_objects("warehouse-script")
print([obj.object_name for obj in objects_script])

[]


# Buckets operation

## Create bucket

Let create buckets name `bronze`, `silver` and `gold` -> using `make_bucket()` method

But I will use my defined function `check_bucket-exist()` instead

In [56]:
new_buckets = ["bronze", "silver", "gold"]
for _ in new_buckets:
    check_bucket_exist(client,_)

bucket bronze exits
bucket silver exits
bucket gold exits


now let check if there were buckets which just be created

In [57]:
print(client.list_buckets())

[Bucket('bronze'), Bucket('gold'), Bucket('silver'), Bucket('warehouse-manual'), Bucket('warehouse-script')]


# Objects operation

## Remove objects

Can see that in warehouse-script, there are 4 objects. Try to delete all objects in warehouse-script.

In [45]:
def delete_all(client, bucket: str):
    try: 
        objects = client.list_objects(bucket)
        for obj in objects:
            client.remove_object(bucket, obj.object_name)
        print("delete all objects")
    except S3Error as e:
        print(e)

In [46]:
delete_all(client, "warehouse-script")

delete all objects


## Get object from bucket

Now try to get data from warehouse-manual

In [8]:
def get_all(client, bucket: str, path: str):
    try:
        # get all objects in the bucket
        objects = client.list_objects(bucket)

        # get all files in bucket and load to path
        for obj in objects:
            client.fget_object(bucket, obj.object_name, f"{path}/{obj.object_name}")
            print(f"downloaded {obj.object_name} done")
        print(f"All following files have been downloaded to {path}")
    except S3Error as e:
        print(e)

Download all file to ./data directory in local machine

In [20]:
get_all(client, "warehouse-manual", "./data")

downloaded 4.PNG done
downloaded Largest-Bank.csv done
downloaded bank_market_cap.json done
downloaded download.jpeg done
All following files have been downloaded to ./data


In [29]:
os.listdir("./data")

['4.PNG', 'bank_market_cap.json', 'Largest-Bank.csv', 'download.jpeg']

Now there are 4 files in ./data directory

## Put object to bucket

Now try to put all data from a local directory to warehouse-script

In [40]:
def put_all(client, bucket: str, local_path: str):
    # change workspace to destination directory
    files = os.listdir(local_path)
    
    #put all objects to bucket
    for file in files:
        client.fput_object(bucket, file, f"{local_path}/{file}")
        print(f"uploaded {file} done")
    print(f"All following files have been uploaded to {bucket}")

There are 4 files in ./data directory, now put these to warehouse-script

In [49]:
put_all(client, "warehouse-script", "./data")


uploaded 4.PNG done
uploaded bank_market_cap.json done
uploaded Largest-Bank.csv done
uploaded download.jpeg done
All following files have been uploaded to warehouse-script


Lets check was it successful in warehouase-script

In [50]:
objects = [obj.object_name for obj in client.list_objects("warehouse-script")]
print(objects)

['4.PNG', 'Largest-Bank.csv', 'bank_market_cap.json', 'download.jpeg']
